In [218]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import mat73

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [219]:
# define constants and helper functions

# file to process
fname = "indy_20160407_02.mat"

# bin width
bin_width = 16e-3

# trial and error to match value in refh_results.csv
min_neuron_len = 383
    
# define a valid neuron
def valid_neuron(neuron):
    return len(neuron) >= min_neuron_len

# compute snr same as in Makin 2018
def compute_snr(r2):
    return -10*np.log10(1-r2)

In [220]:
# load and preprocess data

# load data from a single file into python
data = mat73.loadmat(fname)

# extract variables from the data dictionary
for key,val in data.items():
    exec(key + '=val')

# merge (flatten) the spikes data structure for ease of manipulation
spks = []
for electrode in spikes:
    for neuron in electrode:
        if valid_neuron(neuron):
            spks.append(neuron)

n_neurons = len(spks)

In [221]:
# downsample kinematic data (4ms -> 16ms) and time vector

pos  = cursor_pos[ 3:-1:4 , : ] # start at sample 3 - makes the vector same size as in refh_results.csv
t_d  = t[ 3:-1:4 ] 

In [222]:
# bin the spike data

bins = np.append( t_d , t_d[-1]+bin_width )
binned_spikes = np.zeros( (n_neurons,len(t_d)) )

for i,neuron in enumerate(spks):
    cnt,_ = np.histogram(neuron,bins)
    binned_spikes[i,:] = cnt

In [223]:
# divide into training and testing data sets

t_train_sec     = 320
n_train_samples = int(t_train_sec / bin_width)

pos_train           = pos[ :n_train_samples , : ]
pos_test            = pos[ n_train_samples: , : ]

binned_spikes_train = binned_spikes[ : , :n_train_samples ]
binned_spikes_test  = binned_spikes[ : , n_train_samples: ]

t_train             = t_d [ :n_train_samples ]
t_test              = t_d [ n_train_samples: ]

In [224]:
# verify sizes are as expected

print(f"binned_spikes_train: {binned_spikes_train.shape}")
print(f"pos_train: {pos_train.shape}")
print("")
print(f"binned_spikes_test: {binned_spikes_test.shape}")
print(f"pos_test: {pos_test.shape}")

binned_spikes_train: (291, 20000)
pos_train: (20000, 2)

binned_spikes_test: (291, 31111)
pos_test: (31111, 2)


In [230]:
data1 = binned_spikes_train[0].reshape(-1,1)
data2x = np.transpose(pos_train)[0] # comparing to the monkey's finger x position
data2y = np.transpose(pos_train)[1] # comparing to the monkey's finger y position

#data3 = binned_spikes_test[0].reshape(-1,1)
#data4 = np.transpose(pos_test)[0]

#print(data1[0:320].shape)
#print(data2[0:320].shape)
#print(data2y)


[58.55611041 58.55615266 58.55624154 ... 68.13437872 68.06021646
 67.90441536]


In [226]:
# Trying out LinearRegression() model

def lregModel(data1, data2):
    lreg = LinearRegression().fit(data1, data2)
    print(lreg)

    # Obtain the coefficient of determination by calling the model with the score() function, then print the coefficient:
    r_sq = lreg.score(data1, data2x)
    print('coefficient of determination:', r_sq)

    # Print the Intercept:
    print('intercept:', lreg.intercept_)

    # Print the Slope:
    print('slope:', lreg.coef_)

    # Predict a Response and print it:
    data_pred = lreg.predict(data1)
    print('Predicted response:', data_pred, sep='\n')
    print('Length of prediction: ', len(data_pred))
    print('SNR: ', compute_snr(r_sq))
    
    print('R^2 using r2_score: ', r2_score(data2, data_pred))

In [227]:
print("Linear regression of neurons firing to Monkey's finger x position:\n")
lregModel(data1, data2x)

print("\n\n\nLinear regression of neurons firing to Monkey's finger y position:\n")
lregModel(data1, data2y)



Linear regression of neurons firing to Monkey's finger x position:

LinearRegression()
coefficient of determination: 0.0039943023709970715
intercept: -32.85649670590573
slope: [-10.02762025]
Predicted response:
[-42.88411695 -32.85649671 -32.85649671 ... -42.88411695 -32.85649671
 -32.85649671]
Length of prediction:  20000
SNR:  0.017381771970257916
R^2 using r2_score:  0.0039943023709970715



Linear regression of neurons firing to Monkey's finger y position:

LinearRegression()
coefficient of determination: -3.583731737678807
intercept: 62.23491066614677
slope: [-6.45093757]
Predicted response:
[55.7839731  62.23491067 62.23491067 ... 55.7839731  62.23491067
 62.23491067]
Length of prediction:  20000
SNR:  -6.612191926733506
R^2 using r2_score:  0.006535524747832477
